In [491]:
import pdfplumber
import pandas as pd
import os
#import spacy 
import numpy as np
from IPython.display import clear_output
import re

In [494]:
def get_names(dirname):
    """
    Get employee names, store ids and filenames
    """
    employee_names = []
    store_ids = []
    filepaths = []
    uids = []
    for root, dirs, files in os.walk(dirname):
        for item in files:
            if ".xlsx" in item:
                skip=True
                continue
            else:
                filename = os.path.join(root, item)
                filepaths.append(filename)
                item = re.sub('[\s]+', ' ', 
                              re.sub('^\s\-\s', ' - ', 
                                     re.sub('[\(\)]+', ' - ', item)))
                item = re.sub(' - not formatted', ' ', item)
                item = re.sub('(\s\-)[A-Z0-9]', ' - ', item)
                #item = re.sub('[A-Z0-9](\-\s)[A-Z0-9]', ' - ', item)
                item = re.sub('.pdf', ' ', item)
                item = re.sub('.rtf', ' ', item)
                item = re.sub('.doc[x]*', ' ', item)
                item = re.sub('[\s]+', ' ', item)
                #id = re.findall('[A-Z0-9]{3}', item)
                splits = item.split(" - ")
                employee_name = splits[0]
                #print(employee_name)
                #print(item)
                if len(splits) != 1:
                    #print(item)
                    store_id = splits[1]
                    if len(splits) == 3:
                        uid = splits[2]
                    else: 
                        uid = ''
                        print(employee_name, "don't have uid")
                    #print(store_id)
                else: 
                    store_id = ''
                    print(employee_name, "don't have store id")
                #print(uid)
                employee_names.append(employee_name)
                store_ids.append(store_id) 
                uids.append(uid)
    #print("get names: finished")
    return employee_names, store_ids, filepaths, uids

In [495]:
def convert_to_table(dirname):
    dirpath = dirname
    employee_names, store_ids, filepaths, uids = get_names(dirpath)
    all_dict = {"employee_name":[], "store":[], "uid":[]}
    for i in range(len(filepaths)):
        try:
            clear_output(wait=True)
            employee_name = employee_names[i]
            store_id = store_ids[i]
            uid = uids[i]
            path = filepaths[i]
            all_dict["employee_name"].append(employee_name)
            all_dict["store"].append(store_id)
            all_dict["uid"].append(uid)
            print("Current progress:", i, "/", len(filepaths)-1, np.round((i+1)/len(filepaths)*100, 2), "%")
            print("finished", employee_name)
        except TypeError:
            continue
    return pd.DataFrame(all_dict)

In [496]:
dirpath = "../Career Builder (Resumes)/"
df = convert_to_table(dirpath)

Current progress: 538 / 538 100.0 %
finished Yan, Xu


In [497]:
# one abnormal name, change by hand 
df[df.employee_name=="Muhoza"]
df.iloc[293, 0] = "Muhoza Juste"
df.iloc[293,1] = "WW 353"
df.iloc[293,2] = "N5M"

- There are 6 names unmatched to the name in new_hire file -> change them manually for matching 

In [502]:
# Akash, Sadana -> Sadana, Akash
df.iloc[12, 0] = "Sadana, Akash"
# Bell, Sarah -> Ball, Sarah
df.iloc[59, 0] = "Ball, Sarah"
# Mcgregor, Ethan -> McGregor, Ethan
df.iloc[267, 0] = "McGregor, Ethan"
# Nasir, Mohammad -> Nasir, Mohammad Humayon
df.iloc[303, 0] = "Nasir, Mohammad Humayon"
# Sohi, Jaipal -> Sohi, Jaipal Singh
df.iloc[411, 0] = "Sohi, Jaipal Singh"

In [503]:
df[df.employee_name=="Bois, Kevin"]
df.iloc[68, 0] = "Bois, Dan Kevin"
df.iloc[68, 1] = "TB 421"
df.iloc[68, 2] = "MVT"

In [504]:
len(df)

539

# Then join new_hire table for uid 

In [505]:
has_resumes = df.copy()

In [506]:
new_hire = pd.read_excel("../1 - New Hires for CB Resumes Pull.xlsx")
new_hire_modified = new_hire.copy()

In [507]:
def remove_last_space(string):
    if string !="" and string[-1] == ' ':
        string = string[:-1]
    if string !="" and string[-3:] == 'FRE':
        string = string[:-4]
    return string 

In [508]:
def get_storeid(text):
    """
    Parse store_id for matching 
    """
    try:
        parse = re.match('([A-Z]{2})\s[a-zA-Z\séè\'\.\-]+([0-9]+)', text)
        banner = parse.group(1)
        code = parse.group(2)
        store_id = banner + " " + code
        return store_id
    except AttributeError:
        print(text)
        return ""

In [509]:
has_resumes.store = has_resumes.store.astype(str)
has_resumes.store = has_resumes.store.apply(lambda x: remove_last_space(x))
has_resumes.employee_name = has_resumes.employee_name.apply(lambda x: remove_last_space(x))

In [510]:
new_hire_modified['store'] = new_hire.Store.apply(lambda x: get_storeid(x))
new_hire_modified = new_hire_modified.drop(columns=["Store"])
new_hire_modified.rename(columns = {'Employee Name (Last, First)': "employee_name"}, inplace=True)
new_hire_modified.employee_name = new_hire_modified.employee_name.apply(lambda x: remove_last_space(x))

In [511]:
df_merged = (pd.merge(has_resumes, new_hire_modified, how="left", on=["employee_name", "store"])).drop(columns=["uid", "Resume Pulled?"])

In [512]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539 entries, 0 to 538
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   employee_name          539 non-null    object 
 1   store                  539 non-null    object 
 2   ADP ID                 524 non-null    float64
 3   3-Digit ID             524 non-null    object 
 4   District               524 non-null    object 
 5   Job Title Description  524 non-null    object 
 6   Hire Date              524 non-null    object 
dtypes: float64(1), object(6)
memory usage: 33.7+ KB


In [513]:
df_merged[df_merged["3-Digit ID"].isnull()]

,employee_name,store,ADP ID,3-Digit ID,District,Job Title Description,Hire Date
46,"Bajwa, Manpreet",WETC 1112,NaN,NaN,NaN,NaN,NaN
104,"Cole, Jason",WE 55,NaN,NaN,NaN,NaN,NaN
128,Engy Ibrahim,TB 433,NaN,NaN,NaN,NaN,NaN
192,"Kablanian, Alexan",TB 463,NaN,NaN,NaN,NaN,NaN
201,"Kaur, Sukhpreet",WW 1157,NaN,NaN,NaN,NaN,NaN
293,Muhoza Juste,WW 353,NaN,NaN,NaN,NaN,NaN
310,"Nishad, Rayees",TB425,NaN,NaN,NaN,NaN,NaN
313,"O'Brien, Ashley",WE 591,NaN,NaN,NaN,NaN,NaN
334,"Primeaux-Kostka, Alex",WW 1168,NaN,NaN,NaN,NaN,NaN
404,"Singh, Satwinder",WE 162,NaN,NaN,NaN,NaN,NaN


- Some employee's store is unmatched to the store recorded in new_hire -> manually modify them for matching 

In [514]:
has_resumes_modified = has_resumes.set_index("employee_name")
new_hire_modified2 = new_hire_modified.set_index("employee_name")

In [515]:
has_resumes_modified.loc["Bajwa, Manpreet","store"] = new_hire_modified2.loc["Bajwa, Manpreet","store"]
has_resumes_modified.loc["Cole, Jason","store"] = new_hire_modified2.loc["Cole, Jason","store"]
has_resumes_modified.loc["Kablanian, Alexan","store"] = new_hire_modified2.loc["Kablanian, Alexan","store"]
has_resumes_modified.loc["Kaur, Sukhpreet","store"] = new_hire_modified2.loc["Kaur, Sukhpreet","store"]
has_resumes_modified.loc["Nishad, Rayees","store"] = new_hire_modified2.loc["Nishad, Rayees","store"]
has_resumes_modified.loc["Primeaux-Kostka, Alex","store"] = new_hire_modified2.loc["Primeaux-Kostka, Alex","store"]
has_resumes_modified.loc["Singh, Satwinder","store"] = new_hire_modified2.loc["Singh, Satwinder","store"]
has_resumes_modified.loc["Zhu, WeiWei","store"] = new_hire_modified2.loc["Zhu, WeiWei","store"]
has_resumes_modified.loc["Boucher, Jeremie","store"] = new_hire_modified2.loc["Boucher, Jeremie","store"]
has_resumes_modified.loc["Thomas, Marc","store"] = new_hire_modified2.loc["Thomas, Marc","store"]
has_resumes_modified.loc["Hayward, Sheldon","store"] = new_hire_modified2.loc["Hayward, Sheldon","store"]
has_resumes_modified.loc["Lirette, Mariah","store"] = new_hire_modified2.loc["Lirette, Mariah","store"]

In [516]:
has_resumes_modified = has_resumes_modified.reset_index()

In [517]:
new_hire_modified2 = new_hire_modified2.reset_index()

# Join again 

In [518]:
df_merged2 = (pd.merge(has_resumes_modified, new_hire_modified2, how="left", on=["employee_name", "store"])).drop(columns=["uid", "Resume Pulled?"])

In [519]:
df_merged2[df_merged2["3-Digit ID"].isnull()]

,employee_name,store,ADP ID,3-Digit ID,District,Job Title Description,Hire Date
128,Engy Ibrahim,TB 433,NaN,NaN,NaN,NaN,NaN
293,Muhoza Juste,WW 353,NaN,NaN,NaN,NaN,NaN
313,"O'Brien, Ashley",WE 591,NaN,NaN,NaN,NaN,NaN


- Engy Ibrahim, Muhoza Juste and O'Brien, Ashley don't exist in new_hire file. 
- employee_code for Engy Ibrahim, Muhoza Juste are downloaded within their resume file name, so I added their employee name by hand. 
- However, O'Brien, Ashley's employee code can't be found.

In [520]:
df_merged2 = df_merged2.set_index("employee_name")

In [521]:
df_merged2.loc["Engy Ibrahim", "3-Digit ID"] = "MWA"
df_merged2.loc["Muhoza Juste", "3-Digit ID"] = "N5M"

In [522]:
df_merged_final = df_merged2.reset_index()

In [523]:
df_merged_final[df_merged_final["3-Digit ID"].isnull()]

,employee_name,store,ADP ID,3-Digit ID,District,Job Title Description,Hire Date
313,"O'Brien, Ashley",WE 591,NaN,NaN,NaN,NaN,NaN


# final table: df_merged_final

In [524]:
df_merged_final.rename(columns={"3-Digit ID":"employee_code"}, inplace=True)

In [525]:
df_merged_final = df_merged_final[["employee_name", "employee_code", "store", "ADP ID", "District", "Job Title Description", "Hire Date"]]

In [526]:
df_merged_final

,employee_name,employee_code,store,ADP ID,District,Job Title Description,Hire Date
0,"Abbasi, Samiee Z",N42,WW 374,25256.0,GTA East,SALES ASSOCIATE,"Thursday, July 18, 2019"
1,"Abdi, Melika",N3O,TB 146,25237.0,BC East,SALES ASSOCIATE,"Thursday, July 11, 2019"
2,"Abdulali, Kaizar",BDP,WW 252,25258.0,Alberta North,SALES MANAGER,"Monday, July 29, 2019"
3,"Abro, Muhammad Wali",N62,WW 394,25331.0,GTA West,SALES ASSOCIATE,"Tuesday, August 6, 2019"
4,"Abrum, Nabil",N7R,WE 540,25404.0,GTA West & Ottawa - ETC,SALES ASSOCIATE,"Wednesday, September 11, 2019"
...,...,...,...,...,...,...,...
534,"Sukhiani, Salman",N03,WW 341,25104.0,GTA Central,SALES ASSOCIATE,"Monday, May 27, 2019"
535,"Tremblay, Shawnee",N3L,WE 556,25239.0,Quebec West - ETC,SALES ASSOCIATE,"Tuesday, July 23, 2019"
536,"Turner, Kyle",MPQ,WW 854,24774.0,Atlantic,SALES ASSOCIATE,"Thursday, February 7, 2019"
537,"Wu, Alexander",LVW,WW 115,25057.0,BC East,SALES ASSOCIATE,"Thursday, May 16, 2019"


# Summary 
1. There are 539 employees having resumes (pdf, doc, docx, rtf).
2. 538/539 employees have unique `employee_code`, only one employee (O'Brien, Ashley) doesn't have matched `employee_code`